## Standalone DF and DataTap Demo

#### -1- Establish A Login Session With The HPE CP Enviroment
#### Initialize the environment:

Let's first define the environment variables according to your Ezmeral Container Platform user account and tenant name, and the Ezmeral Container Platform API system endpoint:

In [ ]:
# DataFabric Variables
#cldb_host="m2-dfse-us.mip.storage.hpecorp.net"
#cldb_endpoint="m2-dfse-us.mip.storage.hpecorp.net:8443"
#cldb_user="mapr"
#cldb_pass="datafab123"

# ECP Variables
username="courtney" # your ECP tenant login credentials - username 
password="Admin123" # your ECP tenant login credentials - password
controller_endpoint="16.0.8.220:8080"
gateway_host="mip-bd-vm207.mip.storage.hpecorp.net"
tenantname="lazy-tenant"

echo "your working context is:" $username-$password-$tenantname

#### -2- Authenticate as a Tenant user in the specified tenant:

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "http://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Quickly check to ensure you can make REST API calls within your tenant working context:
Here you will fetch information about the session you have just established.

In [ ]:
curl -k -s --request GET "http://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | python -m json.tool

### -3- Create a Dtap using Existing DataFabric Volume:

In [ ]:
curl -k -i -s --request POST "http://${controller_endpoint}/api/v1/dataconn" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--header "X-BDS-SESSION: $sessionlocation" \
--data '{
    "bdfs_root": {
        "path_from_endpoint": "/demo-vols/dsr-test1"
    },
    "endpoint": {
        "cldb": [
            "m2-maprse-dev02.mip.storage.hpecorp.net",
            "m2-maprse-dev03.mip.storage.hpecorp.net",
            "m2-maprse-dev04.mip.storage.hpecorp.net"
        ],
        "cluster_name": "m2-dfse-us.mip.storage.hpecorp.net",
        "impersonation_enabled": true,
        "mapr_tenant_volume": false,
        "port": 7222,
        "secure": true,
        "ticket": "se_svc_imp",
        "ticket_type": "servicewithimpersonation",
        "ticket_user": "se_svc",
        "type": "mapr"
    },
    "flags": {
        "read_only": false
    },
    "label": {
        "description": "DSR Test Volume",
        "name": "dsr-demo"
    }
}'

### -4- Create New DataFabric Volume:

In [ ]:
curl -k -X POST "https://m2-dfse-us.mip.storage.hpecorp.net:8443/rest/volume/create?name=netflix_titles&path=/netflix_titles&readAce=g:se_lab_admins&writeAce=g:se_lab_admins" --user mapr:datafab123

### -5- Create DTap to New DataFabric Volume

In [ ]:
curl -k -i -s --request POST "http://${controller_endpoint}/api/v1/dataconn" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--header "X-BDS-SESSION: $sessionlocation" \
--data '{
    "bdfs_root": {
        "path_from_endpoint": "/netflix_titles"
    },
    "endpoint": {
        "cldb": [
            "m2-maprse-dev02.mip.storage.hpecorp.net",
            "m2-maprse-dev03.mip.storage.hpecorp.net",
            "m2-maprse-dev04.mip.storage.hpecorp.net"
        ],
        "cluster_name": "m2-dfse-us.mip.storage.hpecorp.net",
        "impersonation_enabled": true,
        "mapr_tenant_volume": false,
        "port": 7222,
        "secure": true,
        "ticket": "se_svc_imp",
        "ticket_type": "servicewithimpersonation",
        "ticket_user": "se_svc",
        "type": "mapr"
    },
    "flags": {
        "read_only": false
    },
    "label": {
        "description": "netflix titles",
        "name": "netflix"
    }
}'

In [ ]:
export http_proxy=http://web-proxy.corp.hpecorp.net:8080
export https_proxy=http://web-proxy.corp.hpecorp.net:8080
wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-04.csv

In [ ]:
hadoop fs -copyFromLocal netflix_titles.csv.zip dtap://netflix/

In [ ]:
hadoop fs -ls dtap://netflix/